# SciML SANUM2024
# Lab 1: Introduction to Julia

In this course we will use the programming language Julia. This is a modern, compiled,
high-level, open-source language developed at MIT. It is becoming increasingly
important in high-performance computing and AI, including by Astrazeneca, Moderna and
Pfizer in drug development and clinical trial accelleration, IBM for medical diagnosis,
MIT for robot locomotion, and elsewhere.

It is ideal for a course on Scientific Machine Learning (SciML)  because it both allows fast
implementation of algorithms but also has support for fast automatic-differentiation,
a feature that is of importance in machine learning.
Also, the libraries for solving differential equations and SciML are quite advanced.
As a bonus, it is easy-to-read and fun to write.

This first lab introduces the basics of Julia and some of its unique features in terms of creating custom types.
This will become valuable in implementing automatic-differentiation for generic code, a feature that
is particularly useful for SciML as it allows combining neural networks with general time-steppers.

**Learning Outcomes**

1. Creating functions, both named and anonymous.
2. The notion of a type and how to make your own type.
3. Defining functions whose arguments are restricted to specific types.
4. Overloading functions like `+`, `*`, and `exp` for a custom type.
5. Construction of a dense `Vector` or `Matrix` either directly or via comprehensions or broadcasting.

In what follows we need to use the testing package, which provides a macro called `@test`
that error whenever a test returns false. We load this as follows:

In [5]:
using Test

@test 5 == 5

Test Passed

## 1.1 Functions in Julia

We begin with creating functions, which can be done in a number of ways.
The most standard way is using the keyword `function`,
followed by a name for the function,
and in parentheses a list of arguments.
Let's make a function that takes in a
single number $x$ and returns $x^2$.

In [9]:
function sq(x)
    x^2
end

sq(5), sq(5.2), sq(5.0)

(25, 27.040000000000003, 25.0)

There is also a convenient syntax for defining functions on
one line, e.g., we can also write

In [11]:
sq(x) = x^2 # this is exactly the same as above

sq(5)

25

Multiple arguments to the function can be included with `,`.
Here's a function that takes in 3 arguments and returns the average.
(We write it on 3 lines only to show that functions can take multiple lines.)

In [36]:
function av(x, y, z)
    ret = x + y
    ret = ret + z
    ret/3
end

av(1,2,4) # compiled for Int,Int,Int
@time av(1,2,4.3) # re-compile for Int,Int,Float64
@time av(1,2,4.3) # uses pre-compile version for Int,Int,Float64


𝐱 = [1,2,4]

function av(𝐱)
    ret = 𝐱[1] + 𝐱[2]
    ret = ret + 𝐱[3]
    ret/3
end
av(𝐱)
sum(𝐱)/length(𝐱)

function av(𝐱)
    ret = 0
    for k = 1:length(𝐱)
        ret = ret + 𝐱[k]
    end
    ret, ret/length(𝐱) # returns "tuple"
end

av([1,2,3,45,56])

  0.000001 seconds
  0.000000 seconds


5

**Remark**: Julia is a compiled language: the first time we run a command on a specific type it compiles for that type.
The second time it is called on a type it reuses the precompiled function.

 **Warning**: Variables live in different scopes.  In the previous example, `x`, `y`, `z` and `ret` are _local variables_:
they only exist inside of `av`.
If you reference variables not defined inside the function, they will use the outer scope definition.
The following example shows that if we mistype the first argument as `xx`,
then it takes on the outer scope definition `x`, which is a complex number:

In [19]:
function av2(xx, y, z)
    (x + y + z)/3
end

x = 6
av2(2, 2, 2) # uses x = 5 from outside function

3.3333333333333335

You should avoid using global variables:
we should ideally be able to predict the output of a function from knowing just the inputs.
This also leads to faster code as the compiler can know the type of the variable.

Here is an example of a more complicated function that computes
the _(right-sided) rectangular rule_
for approximating an integral: choose $n$ large so that
$$
  ∫_0^1 f(x) {\rm d}x ≈ {1 \over n} ∑_{j=1}^n f(j/n).
$$
It demonstrates that functions can take in  other functions as inputs
and shows the syntax for a for-loop:

In [45]:
function rightrectangularrule(f, n)
    ret = 0.0
    for j = 1:n
        ret = ret + f(j/n)
    end
    ret/n
end

@time rightrectangularrule(exp, 100_000_000) # _ are like commas in numbers

  0.343367 seconds


1.7182818370501693

### Anonymous (lambda) functions

It is possible to make unnamed anonymous functions,
with two variants on syntax:

In [61]:
# in Matlab @(x) exp(x^2)
f = x -> exp(x^2) # makes an "anonymous function"
# What's the difference between this and a named function
# f(x) = exp(x^2)
# The named function is like a global const function

f = (x,y) -> exp(x^2)+cos(y) # makes an "anonymous function"

function returnanotherfunction(k)
    x -> cos(k*x)
end

returnanotherfunction(5)(3)

rightrectangularrule(x -> exp(x^2), 100_000_000)

1.4626517544986326

In [52]:
myfunction(x) = exp(x^2)
myfunction(x,y) = x + cos(y)
myfunction(3), myfunction(2,3)

(8103.083927575384, 1.0100075033995546)

There is not much difference between named and anonymous functions,
both are compiled in the same manner. The only difference is
named functions are in a sense "permanent". One can essentially think of
named functions as "global constant anonymous functions".

----

**Problem 1(a)** Complete the following function `leftrectangularrule(f, n)` That approximates
an integral using the left-sided rectangular rule:
$$
  ∫_0^1 f(x) {\rm d}x ≈ {1 \over n} ∑_{j=0}^{n-1} f(j/n).
$$

In [ ]:
using Test # Loads `@test` again in case you didn't run the line above.

function leftrectangularrule(f, n)
    # TODO: return (1/n) * ∑_{j=0}^{n-1} f(j/n) computed using a for-loop

end

@test leftrectangularrule(exp, 1000) ≈ exp(1) - 1 atol=1E-3 # tests that the approximation is accurate to 3 digits after the decimal point

**Problem 1(b)** Use an anonymous function as input for `lefrectangularrule` to approximate
the integral of $\cos x^2$ on $[0,1]$ to 5 digits.

In [ ]:
# TODO: use an anonymous function to represent the function cos(x^2) and approximate its integral.

----

## 1.2 Types in Julia

In compiled languages like Julia everything has a "type". The function `typeof` can be used to determine the type of,
for example, a number.
By default when we write an integer (e.g. `-123`) it is of type `Int`:

In [65]:
typeof(5), typeof(5.0), typeof(5.0f0)

(Int64, Float64, Float32)

On a 64-bit machine this will print `Int64`, where the `64` indicates it is using precisely 64 bits
to represent the number. If we write something with
a decimal point it represents a "real" number, whose storage is of type `Float64`:

In [ ]:
#

This is  a floating point number, and again the `64` indicates it is using precisely
64 bits to represent this number. Note that some operations involving `Int`s return `Float64`s:

In [ ]:
#

It is possible to have functions behave differently depending on the input type.
To do so we can add a restriction denoted `::Int` or `::Float64` to the function "signature".
Here we create a function `foo` that is equal to `1` if the input is an `Int`, `0` if the input is
a `Float64`, and `-1` otherwise:

In [68]:
foo(x::Int) = 1
foo(x::Float64) = 0
foo(x) = -1

foo(5), foo(5.0), foo("hi")

(1, 0, -1)

The last line returns a list of `Int`s, which has the type `Tuple`.
Note that there is a difference between an "integer" and the type `Int`: whilst 3.0 is an integer
its type is `Float64` so `foo(3.0) == 0`.

Types allow for combining multiple numbers (or instances of other types) to represent a more complicated
object. A simple example of this is a complex number,
which stores two real numbers $x$ and $y$ (either `Int` or `Float64` or indeed other real number types not yet discussed)
to represent the complex number $x + {\rm i} y$. In Julia ${\rm i} = \sqrt{-1}$ is denoted `im` and
hence we can create a complex number like $1+2{\rm i}$ as follows:

In [73]:
z = 1 + im * 3.3
typeof(z)

ComplexF64 (alias for Complex{Float64})

This complex number has two "fields": the real and imaginary part. Accessing the fields is done
using a `.`, here we display the real and imaginary parts as a `Tuple`:

In [76]:
z.re, z.im

(1.0, 3.3)

When we ask  its type we see it is a `Complex{Int}`:

In [ ]:
#

The `{Int}` indicates that each of the fields is an `Int`.
Note we can add, subtract, multiply, or apply functions like `exp` to complex numbers:

In [78]:
exp(z^2 + 3im)

-4.990294266496956e-5 - 8.834697530948195e-6im

### Supertypes

Every type has a "supertype", which is an "abstract type": something you can't make an instance of it.
For example, in the same way that "integers"
are subsets of the "reals" we have that `Int` and `Float64` are subtypes of
`Real`. Which is a subtype of `Number`. Which, as is everything, a subtype of `Any`. We can see this with the
`supertype` function:

In [88]:
@show typeof(5)
@show supertype(Int)
@show supertype(Signed)
@show supertype(Integer)
@show supertype(Real)
@show supertype(Number)
@show supertype(Any)

typeof(5) = Int64
supertype(Int) = Signed
supertype(Signed) = Integer
supertype(Integer) = Real
supertype(Real) = Number
supertype(Number) = Any
supertype(Any) = Any


Any

In [90]:
newfoo(x::Real) = 1
newfoo(x) = -1
newfoo(5), newfoo(5.3), newfoo(Inf), newfoo(1+im)

(1, 1, 1, -1)

-----
**Problem 2(a)** Use `typeof` to determine the type of `1.2 + 2.3im`.

In [ ]:
# TODO: What is the type of `1.2 + 2.3im`?

**Problem 2(b)** Add another implementation of `foo` that returns `im` if the input
is a `ComplexF64`.

In [ ]:
# TODO: Overload foo for when the input is a ComplexF64 and return im


@test foo(1.1 + 2im) == im

**Problem 3** Consider the Taylor series approximation to the exponential:
$$
\exp z ≈ ∑_{k=0}^n {z^k \over k!}
$$
Complete the function `exp_t(z, n)` that computes this and returns a
`Complex{Float64}` if the input is complex and a `Float64` if the input is real.
Do not use the inbuilt `factorial` function.
Hint: It might help to think inductively: for $s_k = z^k/k!$ we have
$$
  s_{k+1}  = {z \over k+1} s_k.
$$

In [ ]:
function exp_t(z, n)
    # TODO: Compute the first (n+1)-terms of the Taylor series of exp
    # evaluated at z

end

@test exp_t(1.0, 10) isa Float64 # isa is used to test the type of a result
@test exp_t(im, 10) isa ComplexF64 # isa is used to test the type of a result

@test exp_t(1.0, 100) ≈ exp(1)

------

### Making our own Types

One of the powerful parts of Julia is that it's very easy to make our own types. Lets begin with a simple
implementation of a rational function $p/q$ where $p$ and $q$ are `Int`s.  Thus we want to create a new
type called `Rat` with two fields `p` and `q` to represent the numerator and denominator, respectively.
(For simplicity  we won't worry about restricting $p$ and $q$ to be `Int`.)
We can construct such a type using the `struct` keyword:

In [109]:
y = (@show "hi");
y

"hi" = "hi"


"hi"

In [121]:
function foo()
    ret = 0
    for j = 1:5
        ret = ret + j
    end
end
x = foo();
typeof(x)

Nothing

In [91]:
struct Rat # represents p/q
    p # first field is called p
    q # second field is called q
end

A new instance of `Rat` is created via e.g. `Rat(1, 2)` represents 1/2
where the first argument specifies `p` and the second argument `q`.
The fields are accessed by `.`:

In [94]:
x = Rat(1, 2)
x.p, x.q

(1, 2)

Unfortunately we can't actually do anything with this type, yet:

In [95]:
x + x

LoadError: MethodError: no method matching +(::Rat, ::Rat)

[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:587[24m[39m


The error is telling us to overload the `+` function when the inputs are both `Rat`.
To do this we need to "import" the `+` function and then we can overload it like any
other function:

In [99]:
import Base: + # need to import + since its a special builtin function

function +(x::Rat, y::Rat)
    p,q = x.p, x.q # x represents p/q
    r,s = y.p, y.q # y represents r/s
    # x + y == (ps)/(qs) + (rq)/(qs) = (ps + rq)/qs
    Rat(p*s + r*q, q*s)
end

x = Rat(1, 2) # represents 1/2
y = Rat(2, 3) # represents 2/3


x + y

Rat(7, 6)

We can support mixing `Rat` and `Int` by adding additional functionality:

In [103]:
Rat(p::Int) = Rat(p,1)
+(x::Rat, y::Int) = x + Rat(y)

x + 3

Rat(7, 2)

-----

**Problem 4** Support `*`, `-`, `/`, and `==` for `Rat` and `Int`.

In [105]:
? div

search: div divrem DivideError splitdrive code_native @code_native



```
div(x, y)
÷(x, y)
```

The quotient from Euclidean (integer) division. Generally equivalent to a mathematical operation x/y without a fractional part.

See also: [`cld`](@ref), [`fld`](@ref), [`rem`](@ref), [`divrem`](@ref).

# Examples

```jldoctest
julia> 9 ÷ 4
2

julia> -5 ÷ 3
-1

julia> 5.0 ÷ 2
2.0

julia> div.(-5:5, 3)'
1×11 adjoint(::Vector{Int64}) with eltype Int64:
 -1  -1  -1  0  0  0  0  0  1  1  1
```

---

```
div(x, y, r::RoundingMode=RoundToZero)
```

The quotient from Euclidean (integer) division. Computes `x / y`, rounded to an integer according to the rounding mode `r`. In other words, the quantity

```
round(x / y, r)
```

without any intermediate rounding.

!!! compat "Julia 1.4"
    The three-argument method taking a `RoundingMode` requires Julia 1.4 or later.


See also [`fld`](@ref) and [`cld`](@ref), which are special cases of this function.

!!! compat "Julia 1.9"
    `RoundFromZero` requires at least Julia 1.9.


# Examples:

```jldoctest
julia> div(4, 3, RoundDown) # Matches fld(4, 3)
1
julia> div(4, 3, RoundUp) # Matches cld(4, 3)
2
julia> div(5, 2, RoundNearest)
2
julia> div(5, 2, RoundNearestTiesAway)
3
julia> div(-5, 2, RoundNearest)
-2
julia> div(-5, 2, RoundNearestTiesAway)
-3
julia> div(-5, 2, RoundNearestTiesUp)
-2
julia> div(4, 3, RoundFromZero)
2
julia> div(-4, 3, RoundFromZero)
-2
```


In [ ]:
# We import `+`, `-`, `*`, `/` so we can "overload" these operations
# specifically for `Rat`.
import Base: +, -, *, /, ==

# The ::Rat means the following version of `==` is only called if both
# arguments are Rat.
function ==(x::Rat, y::Rat)
    # TODO: implement equality, making sure to check the case where
    # the numerator/denominator are possibly reducible
    # Hint: gcd and div may be useful. Use ? to find out what they do


end

# We can also support equality when `x isa Rat` and `y isa Int`
function ==(x::Rat, y::Int)
    # TODO: implement

end

# TODO: implement ==(x::Int, y::Rat)


@test Rat(1, 2) == Rat(2, 4)
@test Rat(1, 2) ≠ Rat(1, 3)
@test Rat(2,2) == 1
@test 1 == Rat(2,2)

# TODO: implement +, -, *, and /,


@test Rat(1, 2) + Rat(1, 3) == Rat(5, 6)
@test Rat(1, 3) - Rat(1, 2) == Rat(-1, 6)
@test Rat(2, 3) * Rat(3, 4) == Rat(1, 2)
@test Rat(2, 3) / Rat(3, 4) == Rat(8, 9)

## 1.3 Arrays

One can create arrays in multiple ways. For example, the function `zeros(Int, 10)` creates
a 10-element `Vector` whose entries are all `zero(Int) == 0`. Or `fill(x, 10)` creates a
10-element `Vector` whose entries are all equal to `x`. Or you can use a comprehension:
for example `[k^2 for k = 1:10]` creates a vector whose entries are `[1^2, 2^2, …, 10^2]`.
This also works for matrices: `zeros(Int, 10, 5)` creates a 10 × 5 matrix of all zeros,
and `[k^2 + j for k=1:3, j=1:4]` creates the following:

In [128]:
import Base: zero
zero(::Type{Rat}) = Rat(0,1)

r = zeros(5) # default to Float64
r = zeros(Int, 6)
r = zeros(Rat, 3)

r[2] = Rat(1,2)
r

r[3] = 3.4 # cannot set a vector of Rats to a Float64.

LoadError: MethodError: [0mCannot `convert` an object of type [92mFloat64[39m[0m to an object of type [91mRat[39m

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:84[24m[39m
[0m  Rat(::Any, [91m::Any[39m)
[0m[90m   @[39m [36mMain[39m [90m[4mIn[91]:2[24m[39m


In [137]:
[k for k=1:3,j=1:1]

3×1 Matrix{Int64}:
 1
 2
 3

In [136]:
[1,2,3] # Vector with 3 entries

3-element Vector{Int64}:
 1
 2
 3

In [132]:
[j/k for k=2:10, j=1:2]

9×2 Matrix{Float64}:
 0.5       1.0
 0.333333  0.666667
 0.25      0.5
 0.2       0.4
 0.166667  0.333333
 0.142857  0.285714
 0.125     0.25
 0.111111  0.222222
 0.1       0.2

Note sometimes it is best to create a vector/matrix and populate it. For example, the
previous matrix could also been constructed as follows:

In [143]:
A = zeros(Float64, 9, 2, 3)
for k = 1:size(A,1), j = 1:size(A,2), l=1:size(A,3)
    A[k,j,l] = j/(k+1)+l
end
A

9×2×3 Array{Float64, 3}:
[:, :, 1] =
 1.5      2.0
 1.33333  1.66667
 1.25     1.5
 1.2      1.4
 1.16667  1.33333
 1.14286  1.28571
 1.125    1.25
 1.11111  1.22222
 1.1      1.2

[:, :, 2] =
 2.5      3.0
 2.33333  2.66667
 2.25     2.5
 2.2      2.4
 2.16667  2.33333
 2.14286  2.28571
 2.125    2.25
 2.11111  2.22222
 2.1      2.2

[:, :, 3] =
 3.5      4.0
 3.33333  3.66667
 3.25     3.5
 3.2      3.4
 3.16667  3.33333
 3.14286  3.28571
 3.125    3.25
 3.11111  3.22222
 3.1      3.2

**Remark** Julia uses 1-based indexing where the first index of a vector/matrix
is 1. This is standard in all mathematical programming languages (Fortran, Maple, Matlab, Mathematica)
whereas those designed for computer science use 0-based indexing (C, Python, Rust).

Be careful: a `Matrix` or `Vector` can only ever contain entries of the right
type. It will attempt to convert an assignment to the right type but will throw
an error if not successful:

In [ ]:
#

------
**Problem 5(a)** Create a 5×6 matrix whose entries are `Int` which is
one in all entries. Hint: use a for-loop, `ones`, `fill`, or a comprehension.

In [ ]:
# TODO: Create a matrix of ones, 4 different ways

**Problem 5(b)** Create a 1 × 5 `Matrix{Int}` with entries `A[k,j] = j`. Hint: use a for-loop or a comprehension.

In [ ]:
# TODO: Create a 1 × 5  matrix whose entries equal the column, 2 different ways

-------
### Transposes and adjoints

We can also transpose a matrix `A` via `transpose(A)`
or compute the adjoint (conjugate-transpose) via `A'` (which is
equivalent to a transpose when the entries are real).
This is done _lazily_: they return custom types `Transpose` or
`Adjoint` that just wrap the input array and reinterpret the entries.
Here is a simple example:

In [150]:
A = randn(2,3)
B = A'

3×2 adjoint(::Matrix{Float64}) with eltype Float64:
 2.27007   -0.10729
 0.050646  -0.698105
 0.989368   0.0979213

In [152]:
B[2,1] = 2
B
A

2×3 Matrix{Float64}:
  2.27007   2.0       0.989368
 -0.10729  -0.698105  0.0979213

In [153]:
typeof(B)

LinearAlgebra.Adjoint{Float64, Matrix{Float64}}

In [180]:
function foo!(A) # Julia is pass by reference. 
    # ! is like a letter in a function name
    # convention is functions end in ! if they modify their input
    A[1,2] = 3
    A
end

function foo(A) # Julia is pass by reference. 
    B = Array(A) # makes a copy, or Matrix(A)
    foo!(B)
end


A = randn(2,3)
foo(A)

2×3 Matrix{Float64}:
  0.311242    3.0       -0.0338389
 -0.0383633  -0.985443   0.283443

In [181]:
A' # just wraps A in another type
Matrix(A') # makes new matrix with same entries as A'

3×2 Matrix{Float64}:
  0.311242   -0.0383633
 -0.177781   -0.985443
 -0.0338389   0.283443

In [176]:
x = 1:3 # this is like a vector
supertype(supertype(supertype(supertype(typeof(x)))))

AbstractVector{Int64} (alias for AbstractArray{Int64, 1})

In [179]:
copy(x) # For immutable types, copy just returns the same thing

1:3

In [178]:
x[2] = 3

LoadError: CanonicalIndexError: setindex! not defined for UnitRange{Int64}

In [185]:
typeof(A')

LinearAlgebra.Adjoint{Float64, Matrix{Float64}}

In [ ]:
copy(

In [158]:
foo!(A)

2×3 Matrix{Float64}:
  2.27007   3.0       0.989368
 -0.10729  -0.698105  0.0979213

In [159]:
A

2×3 Matrix{Float64}:
  2.27007   3.0       0.989368
 -0.10729  -0.698105  0.0979213

If we change entries of `A'` it actually changes entries of `A` too since
they are pointing to the same locations in memory, just interpreting the data differently:

In [ ]:
#

Note vector adjoints/transposes behave differently than 1 × n matrices: they are
more like row-vectors. For example the following computes the dot product of two vectors:

In [ ]:
#

### Broadcasting

_Broadcasting_ is a powerful and convenient way to create matrices or vectors,
where a function is applied to every entry of a vector or matrix.
By adding `.` to the end of a function we "broadcast" the function over
a vector:

In [197]:
x = [1,2,3]
f = x -> exp(x^2)
f.(x) # [exp(x[k]) for k = 1:length(x)]
A = [1 2; 3 4; 5 6]
exp.(A)

3×2 Matrix{Float64}:
   2.71828    7.38906
  20.0855    54.5982
 148.413    403.429

Broadcasting has some interesting behaviour for matrices.
If one dimension of a matrix (or vector) is `1`, it automatically
repeats the matrix (or vector) to match the size of another example.
In the following we use broadcasting to pointwise-multiply a column and row
vector to make a matrix:

In [201]:
f = (x,y) -> exp(x + cos(y))
A
B = randn(3,2)
f.(A, B)

3×2 Matrix{Float64}:
   3.84854   12.339
  18.1968   113.854
 347.191    900.922

In [202]:
f(A[1,1], B[1,1]) # first entry in f.(A, B)

3.848536018476063

In [206]:
x = [2,3,4]
f.(A, x) # it is the same as f.(A, [x x])

3×2 Matrix{Float64}:
  1.79293    4.8737
  7.46337   20.2876
 77.1967   209.842

In [211]:
y = [1,2]
y'
f.(x, y') # same as f.([x x], [y'; y'; y']) but without making extra matrices

3×2 Matrix{Float64}:
 12.6835   4.8737
 34.4773  13.2481
 93.7191  36.012

In [214]:
# infix operations: +, -, *. Broadcasting put the . before the name of the function
@show x
@show y
x .* y'

x = [2, 3, 4]
y = [1, 2]


3×2 Matrix{Int64}:
 2  4
 3  6
 4  8

Since `size([1,2,3],2) == 1` it repeats the same vector to match the size
`size([4,5]',2) == 2`. Similarly, `[4,5]'` is repeated 3 times. So the
above is equivalent to:

In [ ]:
#

Note we can also use matrix broadcasting with our own functions:

In [ ]:
#

### Ranges

_Ranges_ are another useful example of vectors, but where the entries are defined "lazily" instead of
actually created in memory.
We have already seen that we can represent a range of integers via `a:b`. Note we can
convert it to a `Vector` as follows:

In [216]:
Vector(2:6)

5-element Vector{Int64}:
 2
 3
 4
 5
 6

We can also specify a step:

In [217]:
Vector(2:2:6)

3-element Vector{Int64}:
 2
 4
 6

Finally, the `range` function gives more functionality, for example, we can create 4 evenly
spaced points between `-1` and `1`:

In [221]:
Vector(range(-1, 1; length=5))

5-element Vector{Float64}:
 -1.0
 -0.5
  0.0
  0.5
  1.0

In [230]:
# make a 5 x 1 matrix
range(-1, 1; length=5) .* ones(1,1)
x = reshape(range(-1, 1; length=5), 5, 1) # reshape is "lazy": it doesn't make a copy by default
x = reshape(Vector(range(-1, 1; length=5)), 5, 1)

[x ;;] # matrix concatenation

5×1 Matrix{Float64}:
 -1.0
 -0.5
  0.0
  0.5
  1.0

In [231]:
cos.(1:5)

5-element Vector{Float64}:
  0.5403023058681398
 -0.4161468365471424
 -0.9899924966004454
 -0.6536436208636119
  0.28366218546322625

Note that `Vector` is mutable but a range is not:

In [ ]:
#

Both ranges `Vector` are subtypes of `AbstractVector`, whilst `Matrix` is a subtype of `AbstractMatrix`.

-----

**Problem 5(c)** Create a vector of length 5 whose entries are `Float64`
approximations of `exp(-k)`. Hint: use a for-loop, broadcasting `f.(x)` notation, or a comprehension.

In [ ]:
# TODO: Create a vector whose entries are exp(-k), 3 different ways

### Linear algebra

Matrix-vector multiplication works as expected because `*` is overloaded:

In [232]:
A = [1 2; 3 4; 5 6]

3×2 Matrix{Int64}:
 1  2
 3  4
 5  6

We can also solve least squares problems using `\`:

In [234]:
x = [1,2]
A*x # matrix multiplication

3-element Vector{Int64}:
  5
 11
 17

In [249]:
b = [1,2,3]
x = A \randn(5)  # least squares via QR

LoadError: DimensionMismatch: arguments must have the same number of rows

In [242]:
import Base: *, zero
*(x::Rat, y::Rat) = Rat(x.p * y.p, x.q * y.q)
zero(::Rat) = Rat(0,1)

zero (generic function with 25 methods)

In [243]:
x = [1,2]
Rat.(A) * Rat.(x)

3-element Vector{Rat}:
 Rat(5, 1)
 Rat(11, 1)
 Rat(17, 1)

When a matrix is square, `\` reduces to a linear solve.

In [250]:
randn(2,3) .* randn(3,4)

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 2 and 3

In [252]:
randn(2,1,1) .* randn(1,1,5)

2×1×5 Array{Float64, 3}:
[:, :, 1] =
 -0.6781096312993325
  0.3956462898785999

[:, :, 2] =
  0.10025060216097663
 -0.05849169068294429

[:, :, 3] =
  0.7875911122350363
 -0.4595237806902667

[:, :, 4] =
 -1.3983638537586107
  0.8158820419344779

[:, :, 5] =
 -1.0911115168002214
  0.636614204459308

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*